<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# This block is just for importing the necessary libraries.  
import os
from collections import defaultdict
# Numerical libraries
import pandas as pd
import numpy as np
import biom
import arviz as az
from scipy.spatial.distance import euclidean
# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib_venn import venn2, venn3
# custom utility and plotting functions
from util import (extract_differentials, select_features, 
                  get_genomic_data, collapse_transcripts, 
                  aggregate_pathways,
                  ranking, btest, log_pvalue, read_kegg_dict,
                  ilr_transform_differentials,
                  rename_clades, create_projection,
                  match_all_differentials)
from plot import (rankplot, networkplot, vectorplot)
import random
from scipy.sparse import coo_matrix  
from biom import load_table

# directory paths
amp_directory = '../sfari/data/sra/Combined'
wgs_directory = '../sfari/data/sra_shotgun/Combined'
rna_directory = '../sfari/data/recount3'
kang_directory = '../sfari/data/sra/Kang2017'
sepp_directory = '../sfari/data/sra/Kang2017/deblur/sepp'
results_dir = '../results'
kegg_dir = f'{results_dir}/kegg'
hsa_dir = f'{results_dir}/hsa_kegg'
np.random.seed(0)
random.seed(0)
%matplotlib inline

In [2]:
filename = f'{wgs_directory}/table_func.biom'
func_table = load_table(filename)

func_ids = func_table.ids(axis='observation')
func_df = pd.DataFrame(list(map(lambda x: x.split('|'), func_ids)))                                                   

# convert to sparse matrix for convenience                                                                            
func_df.columns = ['OGU', 'KEGG']                                                                                     
func_df['count'] = 1    

ogus = list(set(func_df['OGU']))  
ogus.sort()
ogu_lookup = pd.Series(np.arange(0, len(ogus)), ogus)                                                            
keggs = list(set(func_df['KEGG']))    
keggs.sort()
kegg_lookup = pd.Series(np.arange(0, len(keggs)), keggs)                                                          
func_df['OGU_id'] = func_df['OGU'].apply(lambda x: ogu_lookup.loc[x]).astype(np.int64)                                    
func_df['KEGG_id'] = func_df['KEGG'].apply(lambda x: kegg_lookup.loc[x]).astype(np.int64)                                 
c, i, j = func_df['count'].values, func_df['OGU_id'].values, func_df['KEGG_id'].values                                
data = coo_matrix((c, (i, j)))                                                                                        
ko_ogu = pd.DataFrame(data.todense(), index=ogus, columns=keggs)    

In [4]:
print(func_df)

            OGU    KEGG  count  OGU_id  KEGG_id
0       1000562  K02793      1       0     1589
1       1000562  K02867      1       0     1641
2       1000562  K02871      1       0     1643
3       1000562  K02874      1       0     1644
4       1000562  K02881      1       0     1650
...         ...     ...    ...     ...      ...
271180   999892  K03665      1    3432     2137
271181   999892  K05991      1    3432     2684
271182   999892  K07106      1    3432     3126
271183   999892  K20370      1    3432     5874
271184   999892  K21949      1    3432     6115

[271185 rows x 5 columns]
